In [1]:
import vtk
import time
import numpy as np

#-------------------------------------------#
#     Qt viewer for UI for angle change     #
#-------------------------------------------#

#PPT on Transformation: http://vtk.org/Wiki/VTK/Presentations
#vtkActor.SetPosition() simply effects where your actor is rendered, but do not modify any coordinates of the underlying polydata physically.  
#SetCenter() apply a vtkTransformFilter to modify point coordinates actually.
#Cube box outline for an actor: https://www.vtk.org/Wiki/VTK/Examples/Cxx/PolyData/Outline
#VTK Bitbucket Examples: https://gitlab.kitware.com/vtk/vtk/tree/master/Examples

# Add the observers to watch for particular events. These invoke Python functions.
Rotating= 0
Panning = 0
Zooming = 0

rotFlagRU = 0 #Flag for rotating right upper arm with interaction with mouse or keyboard
rotFlagRF = 0 #Flag for rotating right fore arm with interaction with mouse or keyboard

# Handle the mouse button events.
def ButtonEvent(obj, event):
    global Rotating, Panning, Zooming, rotFlagRU, rotFlagRF
    if ((event == "LeftButtonPressEvent")):
        Rotating = 1
        #torso.RotateX(30)
        #axes.RotateX(30)
        if (rotFlagRU==1):
            upperArmR.RotateX(30)
        if (rotFlagRF==1):
            upperArmL.RotateX(30)
        renWin.Render()
    elif ((event == "LeftButtonReleaseEvent")):
        Rotating = 0
    elif ((event == "MiddleButtonPressEvent")):
        Panning = 1
        if (rotFlagRU==1):
            upperArmR.RotateZ(30)
        if (rotFlagRF==1):
            upperArmL.RotateZ(30)
        renWin.Render()
    elif ((event == "MiddleButtonReleaseEvent")):
        Panning = 0
    elif ((event == "RightButtonPressEvent")):
        Zooming = 1
        if (rotFlagRU==1):
            upperArmR.RotateY(30)
        if (rotFlagRF==1):
            upperArmL.RotateY(30)
        renWin.Render()
    elif ((event == "RightButtonReleaseEvent")):
        Zooming = 0

# General high-level logic
def MouseMove(obj, event):
    global Rotating, Panning, Zooming
    global iren, renWin, ren
    lastXYpos = iren.GetLastEventPosition()
    lastX = lastXYpos[0]
    lastY = lastXYpos[1]

    xypos = iren.GetEventPosition()
    x = xypos[0]
    y = xypos[1]

    center = renWin.GetSize()
    centerX = center[0]/2.0
    centerY = center[1]/2.0

    if Rotating:
        Rotate(ren, ren.GetActiveCamera(), x, y, lastX, lastY,
               centerX, centerY)
    elif Panning:
        Pan(ren, ren.GetActiveCamera(), x, y, lastX, lastY, centerX,
            centerY)
    elif Zooming:
        Dolly(ren, ren.GetActiveCamera(), x, y, lastX, lastY,
              centerX, centerY)

def Keypress(obj, event):
    global rotFlagRU, rotFlagRF
    key = obj.GetKeySym()
    if key == "e":
        obj.InvokeEvent("DeleteAllObjects")
        sys.exit()
    elif key == "w":
        Wireframe()
    elif key =="s":
        Surface()
    elif key =="u":
        rotFlagRU = 1
        rotFlagRF = 0
    elif key == "f":
        rotFlagRF = 1
        rotFlagRU = 0
    elif key =="n":
        rotFlagRU=0
        rotFlagRF=0

# Routines that translate the events into camera motions.

# This one is associated with the left mouse button. It translates x
# and y relative motions into camera azimuth and elevation commands.
def Rotate(renderer, camera, x, y, lastX, lastY, centerX, centerY):
    camera.Azimuth(lastX-x)
    camera.Elevation(lastY-y)
    camera.OrthogonalizeViewUp()
    renWin.Render()


# Pan translates x-y motion into translation of the focal point and position.
def Pan(renderer, camera, x, y, lastX, lastY, centerX, centerY):
    FPoint = camera.GetFocalPoint()
    FPoint0 = FPoint[0]
    FPoint1 = FPoint[1]
    FPoint2 = FPoint[2]

    PPoint = camera.GetPosition()
    PPoint0 = PPoint[0]
    PPoint1 = PPoint[1]
    PPoint2 = PPoint[2]

    renderer.SetWorldPoint(FPoint0, FPoint1, FPoint2, 1.0)
    renderer.WorldToDisplay()
    DPoint = renderer.GetDisplayPoint()
    focalDepth = DPoint[2]

    APoint0 = centerX+(x-lastX)
    APoint1 = centerY+(y-lastY)

    renderer.SetDisplayPoint(APoint0, APoint1, focalDepth)
    renderer.DisplayToWorld()
    RPoint = renderer.GetWorldPoint()
    RPoint0 = RPoint[0]
    RPoint1 = RPoint[1]
    RPoint2 = RPoint[2]
    RPoint3 = RPoint[3]

    if RPoint3 != 0.0:
        RPoint0 = RPoint0/RPoint3
        RPoint1 = RPoint1/RPoint3
        RPoint2 = RPoint2/RPoint3

    camera.SetFocalPoint((FPoint0-RPoint0)/2.0 + FPoint0,
                          (FPoint1-RPoint1)/2.0 + FPoint1,
                          (FPoint2-RPoint2)/2.0 + FPoint2)
    camera.SetPosition((FPoint0-RPoint0)/2.0 + PPoint0,
                        (FPoint1-RPoint1)/2.0 + PPoint1,
                        (FPoint2-RPoint2)/2.0 + PPoint2)
    renWin.Render()

# Dolly converts y-motion into a camera dolly commands.
def Dolly(renderer, camera, x, y, lastX, lastY, centerX, centerY):
    dollyFactor = pow(1.02,(0.5*(y-lastY)))
    if camera.GetParallelProjection():
        parallelScale = camera.GetParallelScale()*dollyFactor
        camera.SetParallelScale(parallelScale)
    else:
        camera.Dolly(dollyFactor)
        renderer.ResetCameraClippingRange()

    renWin.Render()

# Wireframe sets the representation of all actors to wireframe.
def Wireframe():
    actors = ren.GetActors()
    actors.InitTraversal()
    actor = actors.GetNextItem()
    while actor:
        actor.GetProperty().SetRepresentationToWireframe()
        actor = actors.GetNextItem()

    renWin.Render()

# Surface sets the representation of all actors to surface.
def Surface():
    actors = ren.GetActors()
    actors.InitTraversal()
    actor = actors.GetNextItem()
    while actor:
        actor.GetProperty().SetRepresentationToSurface()
        actor = actors.GetNextItem()
    renWin.Render()

def makeSphere(center, radius, res):
    source = vtk.vtkSphereSource()
    source.SetCenter(center)
    source.SetRadius(radius)
    #source.SetResolution(res)
    # mapper
    mapper = vtk.vtkPolyDataMapper()
    if vtk.VTK_MAJOR_VERSION <= 5:
        mapper.SetInput(source.GetOutput())
    else:
        mapper.SetInputConnection(source.GetOutputPort())
    # actor
    actor = vtk.vtkActor()
    actor.SetMapper(mapper)
    return actor

def makeCyl(ren, rad, height, res, centre):
    # create source
    source = vtk.vtkCylinderSource()
    source.SetCenter(centre)
    source.SetRadius(rad)
    source.SetHeight(height)
    source.SetResolution(res)
    # mapper
    mapper   = vtk.vtkPolyDataMapper()
    cylinder = source.GetOutputPort()
    mapper.SetInputConnection(cylinder)
    
    ###Outline can be added as an option to the argument###
    #outline = vtk.vtkOutlineFilter()
    #outline.SetInputData(source.GetOutput())
    #outlineMapper= vtk.vtkPolyDataMapper()
    #outlineMapper.SetInputConnection(outline.GetOutputPort())
    #outlineActor = vtk.vtkActor()
    #outlineActor.SetMapper(outlineMapper)
    #outlineActor.GetProperty().SetColor(0,0,0)
    #ren.AddActor(outlineActor)
    
    # actor
    actor = vtk.vtkActor()
    actor.SetMapper(mapper)
    return actor

##Showing Axes for Center and Origin for any Actor##
def showAxes(ren, actor):
    transform  = vtk.vtkTransform()
    transform.Translate(actor.GetCenter())  #(25, 0.0, 0.0)
    axesActorCen = vtk.vtkAxesActor()
    axesActorCen.SetUserTransform(transform) 
    axesActorCen.SetTotalLength(20.0, 20.0, 20.0)
    #xAxisLabel = axesActor.GetXAxisCaptionActor2D()
    #xAxisLabel.GetCaptionTextProperty().SetFontSize(6)
    ren.AddActor(axesActorCen)

    transform = vtk.vtkTransform()
    transform.Translate(actor.GetOrigin())
    axesActor = vtk.vtkAxesActor()
    axesActor.SetUserTransform(transform) 
    axesActor.SetTotalLength(20.0, 20.0, 20.0)
    ren.AddActor(axesActor)
    
    return 0

def createLimb(rad, len, rot, translate, origin):
    return 0

In [6]:
def makeSkeleton(ren):
    # assign actor to the renderer
    torsoRad = 10
    torsoLen = 25
    torsoCen = np.array([0, torsoLen/2 , 0])
    torsoOrigin = np.array([0, torsoLen , 0])
    torso    = makeCyl(ren, torsoRad, torsoLen, 100, torsoCen)
    torso.SetOrigin(torsoOrigin)
    ren.AddActor(torso)
    
    uArmRad    = 4
    uArmLen    = 20
    uArmCenR    = torsoOrigin - np.array([(uArmLen/2 + torsoRad), 0, 0])
    uArmOriginR = torsoOrigin - np.array([torsoRad, 0, 0])
    upperArmR   = makeCyl(ren, uArmRad, uArmLen, 100, (0,0,0)) # uArmCen)
    upperArmR.SetOrigin((0,uArmLen/2, 0))
    transform  = vtk.vtkTransform()
    transform.PostMultiply()
    transform.RotateZ(-90)
    transform.Translate(-torsoRad - uArmLen/2, torsoLen, 0)
    upperArmR.SetUserTransform(transform)
    #upperArmR.SetOrigin((-20.0, 25.0, 0.0))
    #upperArm.SetPosition(uArmCen)
    #upperArm.SetOrientation(0,0,-90)
    ren.AddActor(upperArmR)
    
    uArmCenL    = torsoOrigin + np.array([(uArmLen/2 + torsoRad), 0, 0])
    uArmOriginL = torsoOrigin + np.array([torsoRad, 0, 0])
    upperArmL   = makeCyl(ren, uArmRad, uArmLen, 100, (0,0,0))
    upperArmL.SetOrigin((0,uArmLen/2, 0))
    transform  = vtk.vtkTransform()
    transform.PostMultiply()
    transform.RotateZ(90)
    transform.Translate(torsoRad + uArmLen/2, torsoLen, 0)
    upperArmL.SetUserTransform(transform)
    ren.AddActor(upperArmL)
    
    fArmRad    = 2.5
    fArmLen    = 20
    fArmCenR    = torsoOrigin - np.array([(fArmLen/2 + torsoRad), 0, 0])
    fArmOriginR = torsoOrigin - np.array([torsoRad, 0, 0])
    foreArmR  = makeCyl(ren, fArmRad, fArmLen, 100, (0,0,0))
    foreArmR.SetOrigin((0,fArmLen/2, 0))
    transform  = vtk.vtkTransform()
    transform.PostMultiply()
    transform.RotateZ(-90)
    transform.Translate(-torsoRad - uArmLen - fArmLen/2, torsoLen, 0)
    foreArmR.SetUserTransform(transform)
    ren.AddActor(foreArmR)
    
    fArmCenL    = torsoOrigin + np.array([(fArmLen/2 + torsoRad), 0, 0])
    fArmOriginL = torsoOrigin + np.array([torsoRad, 0, 0])
    foreArmL  = makeCyl(ren, fArmRad, fArmLen, 100, (0,0,0))
    foreArmL.SetOrigin((0,fArmLen/2, 0))
    transform  = vtk.vtkTransform()
    transform.PostMultiply()
    transform.RotateZ(90)
    transform.Translate(torsoRad + uArmLen + fArmLen/2, torsoLen, 0)
    foreArmL.SetUserTransform(transform)
    ren.AddActor(foreArmL)
    
    sphereActor = makeSphere((0, torsoLen+7, 0), 7, 100)
    ren.AddActor(sphereActor) #Add head to torso
    
    return sphereActor, torso, upperArmL, upperArmR, foreArmL, foreArmR

def setCamera(ren):
    ren.ResetCamera()                    #Reseting and setting camera angle properties
    cam1 = ren.GetActiveCamera()
    cam1.SetPosition(12.0, 3.0, 150.0)
    cam1.SetFocalPoint(12.0, 12.0, 0.0)
    ren.ResetCameraClippingRange()
    
    return 0

def addInteractor(renWin):
    # Define custom interaction.
    iren = vtk.vtkRenderWindowInteractor()
    iren.SetInteractorStyle(None)
    iren.SetRenderWindow(renWin)

    iren.AddObserver("LeftButtonPressEvent", ButtonEvent)
    iren.AddObserver("LeftButtonReleaseEvent", ButtonEvent)
    iren.AddObserver("MiddleButtonPressEvent", ButtonEvent)
    iren.AddObserver("MiddleButtonReleaseEvent", ButtonEvent)
    iren.AddObserver("RightButtonPressEvent", ButtonEvent)
    iren.AddObserver("RightButtonReleaseEvent", ButtonEvent)
    iren.AddObserver("MouseMoveEvent", MouseMove)
    iren.AddObserver("KeyPressEvent", Keypress)
    
    return iren

def setRenWindow():
    # create a rendering window and renderer
    ren = vtk.vtkRenderer()
    ren.SetBackground(0.1, 0.2, 0.4)     #Add the actors to the renderer, set the background and size
    
    renWin = vtk.vtkRenderWindow()
    renWin.AddRenderer(ren)
    renWin.SetSize(800, 800)
    
    return ren, renWin

In [9]:
ren, renWin = setRenWindow()
sphereActor, torso, upperArmL, upperArmR, foreArmL, foreArmR = makeSkeleton(ren)
showAxes(ren, upperArmR)             #show axes
setCamera(ren)
iren = addInteractor(renWin)

#enable user interface interactor
iren.Initialize()
renWin.Render()
iren.Start()

#time.sleep(1)
#upperArmR.RotateWXYZ(45)
#renWin.Render()
#time.sleep(1)
#upperArmR.RotateX(45)
#renWin.Render()

In [ ]:
#Generally it's better design to build a transform or matrix and then apply that to the
#actor via SetUserTransform() or SetUserMatrix().  That way, the transform or matrix is 
#available to you if you have to move other points or actors between coordinate systems.

#To rotate around a point in space, you have to apply a translation to put that point at 
#the origin, then do the rotation, and then translate back again.

#The actor's Rotate methods are inherited from vtkProp3D, but the code that you
#show is from vtkTransform.

#The actor's RotateWXYZ method commutes the matrices (i.e. multiplies them in
#a different order) as compared to the RotateX,Y,Z methods.  The original author
#might have had a reason for doing this, but I doubt it was a good enough reason
#to justify the confusion that this method causes...
            
#actor->SetUserMatrix(transform->GetMatrix());
#actor->RotateWXYZ(45, 1, 0, 0);
#actor->RotateWXYZ(45, 0, 1, 0); 

print(upperArmR.GetCenter())
print(upperArmR.GetOrigin())
print(upperArmL.GetCenter())
print(upperArmL.GetOrigin())
print(foreArmR.GetCenter())
print(foreArmR.GetOrigin())
print(foreArmL.GetCenter())
print(foreArmL.GetOrigin())
print(torso.GetCenter())
print(torso.GetOrigin())
help(transform)

In [ ]:
class IMU(object):
	def __init__(self, dps=250, gs=2, gyro_bw=100, verbose=False):
		"""
		"""
		self.accel = FXOS8700(gs=gs, verbose=verbose)
		self.gyro = FXAS21002(dps=dps, bw=gyro_bw, verbose=verbose)

	def __del__(self):
		"""
		"""
		pass

	def get(self):
		"""
		"""
		accel, mag = self.accel.get()
		gyro = self.gyro.get()
		return (accel, mag, gyro)

	def getOrientation(self, accel, mag, deg=True):
		ax, ay, az = self.normalize(*accel)
		mx, my, mz = self.normalize(*mag)

		roll = atan2(ay, az)
		pitch = atan2(-ax, ay*sin(roll)+az*cos(roll))

		heading = atan2(
			mz*sin(roll) - my*cos(roll),
			mx*cos(pitch) + my*sin(pitch)*sin(roll) + mz*sin(pitch)*cos(roll)
		)

		if deg:
			roll *= 180/pi
			pitch *= 180/pi
			heading *= 180/pi

			heading = heading if heading >= 0.0 else 360 + heading
			heading = heading if heading <= 360 else heading - 360
		else:
			heading = heading if heading >= 0.0 else 2*pi + heading
			heading = heading if heading <= 2*pi else heading - 2*pi

		return (roll, pitch, heading)

	def normalize(self, x, y, z):
		"""Return a unit vector"""
		norm = sqrt(x * x + y * y + z * z)
		if norm > 0.0:
			inorm = 1/norm
			x *= inorm
			y *= inorm
			z *= inorm
		else:
			raise Exception('division by zero: {} {} {}'.format(x, y, z))
		return (x, y, z)

In [ ]:
import vtk
import time

#
# Next we create an instance of vtkConeSource and set some of its
# properties. The instance of vtkConeSource "cone" is part of a visualization
# pipeline (it is a source process object); it produces data (output type is
# vtkPolyData) which other filters may process.
#
cone = vtk.vtkConeSource()
cone.SetHeight( 3.0 )
cone.SetRadius( 1.0 )
cone.SetResolution( 10 )

#
# In this example we terminate the pipeline with a mapper process object.
# (Intermediate filters such as vtkShrinkPolyData could be inserted in
# between the source and the mapper.)  We create an instance of
# vtkPolyDataMapper to map the polygonal data into graphics primitives. We
# connect the output of the cone source to the input of this mapper.
#
coneMapper = vtk.vtkPolyDataMapper()
coneMapper.SetInputConnection(cone.GetOutputPort())

#
# Create an actor to represent the cone. The actor orchestrates rendering of
# the mapper's graphics primitives. An actor also refers to properties via a
# vtkProperty instance, and includes an internal transformation matrix. We
# set this actor's mapper to be coneMapper which we created above.
#
coneActor = vtk.vtkActor()
coneActor.SetMapper(coneMapper)

#
# Create two renderers and assign actors to them. A renderer renders into a
# viewport within the vtkRenderWindow. It is part or all of a window on the
# screen and it is responsible for drawing the actors it has.  We also set
# the background color here. In this example we are adding the same actor
# to two different renderers; it is okay to add different actors to
# different renderers as well.
#
ren1 = vtk.vtkRenderer()
ren1.AddActor(coneActor)
ren1.SetBackground(0.1, 0.2, 0.4)
ren1.SetViewport(0.0, 0.0, 0.5, 1.0)

ren2 = vtk.vtkRenderer()
ren2.AddActor(coneActor)
ren2.SetBackground(0.1, 0.2, 0.4)
ren2.SetViewport(0.5, 0.0, 1.0, 1.0)

#
# Finally we create the render window which will show up on the screen.
# We add our two renderers into the render window using AddRenderer. We also
# set the size to be 600 pixels by 300.
#
renWin = vtk.vtkRenderWindow()
renWin.AddRenderer( ren1 )
renWin.AddRenderer( ren2 )
renWin.SetSize(600, 300)

#
# Make one camera view 90 degrees from other.
#
ren1.ResetCamera()
ren1.GetActiveCamera().Azimuth(90)

#
# Now we loop over 360 degrees and render the cone each time.
#
for i in range(0,360):
    time.sleep(0.03)

    renWin.Render()
    ren1.GetActiveCamera().Azimuth( 1 )
    ren2.GetActiveCamera().Azimuth( 1 )


In [ ]:
import vtk
from vtk.util.colors import chocolate, mint

# Create implicit function primitives. These have been carefully
# placed to give the effect that we want. We are going to use various
# combinations of these functions to create the shape we want; for
# example, we use planes intersected with a cone (which is infinite in
# extent) to get a finite cone.
cone = vtk.vtkCone()
cone.SetAngle(20)
vertPlane = vtk.vtkPlane()
vertPlane.SetOrigin(.1, 0, 0)
vertPlane.SetNormal(-1, 0, 0)
basePlane = vtk.vtkPlane()
basePlane.SetOrigin(1.2, 0, 0)
basePlane.SetNormal(1, 0, 0)
iceCream = vtk.vtkSphere()
iceCream.SetCenter(1.333, 0, 0)
iceCream.SetRadius(0.5)
bite = vtk.vtkSphere()
bite.SetCenter(1.5, 0, 0.5)
bite.SetRadius(0.25)

# Combine primitives to build ice-cream cone. Clip the cone with planes.
theCone = vtk.vtkImplicitBoolean()
theCone.SetOperationTypeToIntersection()
theCone.AddFunction(cone)
theCone.AddFunction(vertPlane)
theCone.AddFunction(basePlane)

# Take a bite out of the ice cream.
theCream = vtk.vtkImplicitBoolean()
theCream.SetOperationTypeToDifference()
theCream.AddFunction(iceCream)
theCream.AddFunction(bite)

# The sample function generates a distance function from the implicit
# function (which in this case is the cone). This is then contoured to
# get a polygonal surface.
theConeSample = vtk.vtkSampleFunction()
theConeSample.SetImplicitFunction(theCone)
theConeSample.SetModelBounds(-1, 1.5, -1.25, 1.25, -1.25, 1.25)
theConeSample.SetSampleDimensions(60, 60, 60)
theConeSample.ComputeNormalsOff()
theConeSurface = vtk.vtkContourFilter()
theConeSurface.SetInputConnection(theConeSample.GetOutputPort())
theConeSurface.SetValue(0, 0.0)
coneMapper = vtk.vtkPolyDataMapper()
coneMapper.SetInputConnection(theConeSurface.GetOutputPort())
coneMapper.ScalarVisibilityOff()
coneActor = vtk.vtkActor()
coneActor.SetMapper(coneMapper)
coneActor.GetProperty().SetColor(chocolate)

# The same here for the ice cream.
theCreamSample = vtk.vtkSampleFunction()
theCreamSample.SetImplicitFunction(theCream)
theCreamSample.SetModelBounds(0, 2.5, -1.25, 1.25, -1.25, 1.25)
theCreamSample.SetSampleDimensions(60, 60, 60)
theCreamSample.ComputeNormalsOff()
theCreamSurface = vtk.vtkContourFilter()
theCreamSurface.SetInputConnection(theCreamSample.GetOutputPort())
theCreamSurface.SetValue(0, 0.0)
creamMapper = vtk.vtkPolyDataMapper()
creamMapper.SetInputConnection(theCreamSurface.GetOutputPort())
creamMapper.ScalarVisibilityOff()
creamActor = vtk.vtkActor()
creamActor.SetMapper(creamMapper)
creamActor.GetProperty().SetColor(mint)

# Create the usual rendering stuff
ren = vtk.vtkRenderer()
renWin = vtk.vtkRenderWindow()
renWin.AddRenderer(ren)
iren = vtk.vtkRenderWindowInteractor()
iren.SetRenderWindow(renWin)

# Add the actors to the renderer, set the background and size
ren.AddActor(coneActor)
ren.AddActor(creamActor)
ren.SetBackground(1, 1, 1)
renWin.SetSize(250, 250)
ren.ResetCamera()
ren.GetActiveCamera().Roll(90)
ren.GetActiveCamera().Dolly(1.5)
ren.ResetCameraClippingRange()

iren.Initialize()
renWin.Render()
iren.Start()

In [ ]:
import vtk

class Visualizer:
    def visualize(self):
        self.axesActor = vtk.vtkAxesActor()
        self.axesActor.SetXAxisLabelText("X Position")
        self.axesActor.SetYAxisLabelText("Y Position")
        self.axesActor.SetZAxisLabelText("Z Position")

        self.axesActor.SetShaftTypeToLine()
        self.axesActor.SetTotalLength(3.0, 3.0, 3.0)
        self.axesActor.SetPosition(12,0,0)
        self.axesActor.SetNormalizedShaftLength(1.0, 1.0, 1.0)
        self.axesActor.SetNormalizedTipLength(0.05, 0.05, 0.05)

        xAxisLabel = self.axesActor.GetXAxisCaptionActor2D()
        xAxisLabel.GetCaptionTextProperty().SetFontSize(6)

        yAxisLabel = self.axesActor.GetYAxisCaptionActor2D()
        yAxisLabel.GetPositionCoordinate().SetCoordinateSystemToNormalizedViewport()
        yAxisLabel.GetPositionCoordinate().SetValue(-0.20, 0.05)
        yAxisLabel.GetCaptionTextProperty().SetFontSize(6)

        zAxisLabel = self.axesActor.GetZAxisCaptionActor2D()
        zAxisLabel.GetPositionCoordinate().SetCoordinateSystemToNormalizedViewport()
        zAxisLabel.GetPositionCoordinate().SetValue(0.00, -0.13)
        zAxisLabel.GetCaptionTextProperty().SetFontSize(1)

        ren = vtk.vtkRenderer()

        # Create the RenderWindow, Renderer and Actors
        renWin = vtk.vtkRenderWindow()
        renWin.AddRenderer(ren)
        iren = vtk.vtkRenderWindowInteractor()
        iren.SetRenderWindow(renWin)

        # Add the actors to the renderer, set the background and size
        ren.AddActor(self.axesActor)
        ren.SetBackground(0.1, 0.2, 0.4)
        renWin.SetSize(800, 800)

        ren.ResetCamera()
        cam1 = ren.GetActiveCamera()
        cam1.SetPosition(12.0, 3.0, 150.0)
        cam1.SetFocalPoint(12.0, 12.0, 0.0)
        ren.ResetCameraClippingRange()

        iren.Initialize()
        renWin.Render()
        iren.Start()

visualizer = Visualizer()
visualizer.visualize()





In [ ]:
transform = vtk.vtkTransform()
transform.Translate(startPoint)
transform.Concatenate(matrix)
transform.Scale(length, length, length)
 
# Transform the polydata
transformPD = vtk.vtkTransformPolyDataFilter()
transformPD.SetTransform(transform)
transformPD.SetInputConnection(arrowSource.GetOutputPort())
 
#Create a mapper and actor for the arrow
mapper = vtk.vtkPolyDataMapper()
actor = vtk.vtkActor()
 
mapper.SetInputConnection(arrowSource.GetOutputPort())
actor.SetUserMatrix(transform.GetMatrix())
 
actor.SetMapper(mapper)

In [ ]:
import vtk
import random
 
 
'''
There are two alternative ways to apply the transform.
 1) Use vtkTransformPolyDataFilter to create a new transformed polydata. 
    This method is useful if the transformed polydata is needed
      later in the pipeline
    To do this, set USER_MATRIX = True
 2) Apply the transform directly to the actor using vtkProp3D's SetUserMatrix. 
    No new data is produced.
    To do this, set USER_MATRIX = False
'''
USER_MATRIX = False
 
#Create an arrow.
arrowSource = vtk.vtkArrowSource() #vtk.vtkCylinderSource()
 
# Generate a random start and end point
random.seed(8775070)
startPoint = [0 for i in range(3)]
startPoint[0] = random.uniform(-10,10)
startPoint[1] = random.uniform(-10,10)
startPoint[2] = random.uniform(-10,10)
endPoint = [0 for i in range(3)]
endPoint[0] = random.uniform(-10,10)
endPoint[1] = random.uniform(-10,10)
endPoint[2] = random.uniform(-10,10)
 
# Compute a basis
normalizedX = [0 for i in range(3)]
normalizedY = [0 for i in range(3)]
normalizedZ = [0 for i in range(3)]
 
# The X axis is a vector from start to end
math = vtk.vtkMath()
math.Subtract(endPoint, startPoint, normalizedX)
length = math.Norm(normalizedX)
math.Normalize(normalizedX)
 
# The Z axis is an arbitrary vector cross X
arbitrary    = [0 for i in range(3)]
arbitrary[0] = random.uniform(-10,10)
arbitrary[1] = random.uniform(-10,10)
arbitrary[2] = random.uniform(-10,10)
math.Cross(normalizedX, arbitrary, normalizedZ)
math.Normalize(normalizedZ)
 
# The Y axis is Z cross X
math.Cross(normalizedZ, normalizedX, normalizedY)
matrix = vtk.vtkMatrix4x4()
 
# Create the direction cosine matrix
matrix.Identity()
for i in range(3):
  matrix.SetElement(i, 0, normalizedX[i])
  matrix.SetElement(i, 1, normalizedY[i])
  matrix.SetElement(i, 2, normalizedZ[i])
 
# Apply the transforms
transform = vtk.vtkTransform()
transform.Translate(startPoint)
transform.Concatenate(matrix)
transform.Scale(length, length, length)
 
# Transform the polydata
transformPD = vtk.vtkTransformPolyDataFilter()
transformPD.SetTransform(transform)
transformPD.SetInputConnection(arrowSource.GetOutputPort())
 
#Create a mapper and actor for the arrow
mapper = vtk.vtkPolyDataMapper()
actor = vtk.vtkActor()
 
if USER_MATRIX:
    mapper.SetInputConnection(arrowSource.GetOutputPort())
    actor.SetUserMatrix(transform.GetMatrix())
else:
    mapper.SetInputConnection(transformPD.GetOutputPort())
 
actor.SetMapper(mapper)
 
# Create spheres for start and end point
sphereStartSource = vtk.vtkSphereSource()
sphereStartSource.SetCenter(startPoint)
sphereStartMapper = vtk.vtkPolyDataMapper()
sphereStartMapper.SetInputConnection(sphereStartSource.GetOutputPort())
sphereStart = vtk.vtkActor()
sphereStart.SetMapper(sphereStartMapper)
sphereStart.GetProperty().SetColor(1.0, 1.0, .3)
 
sphereEndSource = vtk.vtkSphereSource()
sphereEndSource.SetCenter(endPoint)
sphereEndMapper = vtk.vtkPolyDataMapper()
sphereEndMapper.SetInputConnection(sphereEndSource.GetOutputPort())
sphereEnd = vtk.vtkActor()
sphereEnd.SetMapper(sphereEndMapper)
sphereEnd.GetProperty().SetColor(1.0, .3, .3)
 
#Create a renderer, render window, and interactor
renderer = vtk.vtkRenderer()
renderWindow = vtk.vtkRenderWindow()
renderWindow.AddRenderer(renderer)
renderWindowInteractor = vtk.vtkRenderWindowInteractor()
renderWindowInteractor.SetRenderWindow(renderWindow)
 
#Add the actor to the scene
renderer.AddActor(actor)
renderer.AddActor(sphereStart)
renderer.AddActor(sphereEnd)
renderer.SetBackground(.1, .2, .3) # Background color dark blue
 
 
#Render and interact
renderWindow.Render()
renderWindowInteractor.Start()






In [ ]:
import vtk
 
 
#Create a cube
cube = vtk.vtkSphereSource()
cube.SetRadius(50)
cube.SetThetaResolution(100)
cube.SetPhiResolution(100)
 
cubeMapper = vtk.vtkPolyDataMapper()
cubeMapper.SetInputConnection(cube.GetOutputPort())
 
#create a plane to cut,here it cuts in the XZ direction (xz normal=(1,0,0);XY =(0,0,1),YZ =(0,1,0)
plane = vtk.vtkPlane()
plane.SetOrigin(20, 0, 0)
plane.SetNormal(1, 0, 0)
 
#create cutter
cutter = vtk.vtkCutter()
cutter.SetCutFunction(plane)
cutter.SetInputConnection(cube.GetOutputPort())
cutter.Update()
 
FeatureEdges = vtk.vtkFeatureEdges()
FeatureEdges.SetInputConnection(cutter.GetOutputPort())
FeatureEdges.BoundaryEdgesOn()
FeatureEdges.FeatureEdgesOff()
FeatureEdges.NonManifoldEdgesOff()
FeatureEdges.ManifoldEdgesOff()
FeatureEdges.Update()
 
cutStrips = vtk.vtkStripper() ; #Forms loops (closed polylines) from cutter
cutStrips.SetInputConnection(cutter.GetOutputPort())
cutStrips.Update()
cutPoly = vtk.vtkPolyData() ; #This trick defines polygons as polyline loop
cutPoly.SetPoints((cutStrips.GetOutput()).GetPoints())
cutPoly.SetPolys((cutStrips.GetOutput()).GetLines())
 
cutMapper = vtk.vtkPolyDataMapper()
#cutMapper.SetInput(FeatureEdges.GetOutput())
if vtk.VTK_MAJOR_VERSION <= 5:
    cutMapper.SetInput(cutPoly)
else:
    cutMapper.SetInputData(cutPoly)
 
cutActor = vtk.vtkActor()
cutActor.GetProperty().SetColor(1, 1, 0)
cutActor.GetProperty().SetEdgeColor(0, 1, 0)
 
cutActor.GetProperty().SetLineWidth(2)
cutActor.GetProperty().EdgeVisibilityOn()
##cutActor.GetProperty().SetOpacity(0.7)
cutActor.SetMapper(cutMapper)
 
 
#create renderers and add actors of plane and cube
ren = vtk.vtkRenderer()
ren.AddActor(cutActor)
 
 
#Add renderer to renderwindow and render
renWin = vtk.vtkRenderWindow()
renWin.AddRenderer(ren)
renWin.SetSize(600, 600)
iren = vtk.vtkRenderWindowInteractor()
iren.SetRenderWindow(renWin)
ren.SetBackground(0, 0, 0)
renWin.Render()
iren.Start()

In [ ]:
import vtk
 
x = [
    -1.22396, -1.17188, -1.11979, -1.06771, -1.01562, -0.963542,
    -0.911458, -0.859375, -0.807292, -0.755208, -0.703125, -0.651042,
    -0.598958, -0.546875, -0.494792, -0.442708, -0.390625, -0.338542,
    -0.286458, -0.234375, -0.182292, -0.130209, -0.078125, -0.026042,
    0.0260415, 0.078125, 0.130208, 0.182291, 0.234375, 0.286458,
    0.338542, 0.390625, 0.442708, 0.494792, 0.546875, 0.598958,
    0.651042, 0.703125, 0.755208, 0.807292, 0.859375, 0.911458,
    0.963542, 1.01562, 1.06771, 1.11979, 1.17188]
 
y = [
    -1.25, -1.17188, -1.09375, -1.01562, -0.9375, -0.859375,
    -0.78125, -0.703125, -0.625, -0.546875, -0.46875, -0.390625,
    -0.3125, -0.234375, -0.15625, -0.078125, 0, 0.078125,
    0.15625, 0.234375, 0.3125, 0.390625, 0.46875, 0.546875,
    0.625, 0.703125, 0.78125, 0.859375, 0.9375, 1.01562,
    1.09375, 1.17188, 1.25]
 
z = [
    0, 0.1, 0.2, 0.3, 0.4, 0.5,
    0.6, 0.7, 0.75, 0.8, 0.9, 1,
    1.1, 1.2, 1.3, 1.4, 1.5, 1.6,
    1.7, 1.75, 1.8, 1.9, 2, 2.1,
    2.2, 2.3, 2.4, 2.5, 2.6, 2.7,
    2.75, 2.8, 2.9, 3, 3.1, 3.2,
    3.3, 3.4, 3.5, 3.6, 3.7, 3.75,
    3.8, 3.9]
 
# Create a rectilinear grid by defining three arrays specifying the
# coordinates in the x-y-z directions.
xCoords = vtk.vtkFloatArray()
for i in x:
    xCoords.InsertNextValue(i)
 
yCoords = vtk.vtkFloatArray()
for i in y:
    yCoords.InsertNextValue(i)
 
zCoords = vtk.vtkFloatArray()
for i in z:
    zCoords.InsertNextValue(i)
 
# The coordinates are assigned to the rectilinear grid. Make sure that
# the number of values in each of the XCoordinates, YCoordinates,
# and ZCoordinates is equal to what is defined in SetDimensions().
#
rgrid = vtk.vtkRectilinearGrid()
rgrid.SetDimensions(len(x), len(y), len(z))
rgrid.SetXCoordinates(xCoords)
rgrid.SetYCoordinates(yCoords)
rgrid.SetZCoordinates(zCoords)
 
# Extract a plane from the grid to see what we've got.
plane = vtk.vtkRectilinearGridGeometryFilter()
plane.SetInputData(rgrid)
plane.SetExtent(0, 46, 16, 16, 0, 43)
 
rgridMapper = vtk.vtkPolyDataMapper()
rgridMapper.SetInputConnection(plane.GetOutputPort())
 
wireActor = vtk.vtkActor()
wireActor.SetMapper(rgridMapper)
wireActor.GetProperty().SetRepresentationToWireframe()
wireActor.GetProperty().SetColor(0, 0, 0)
 
# Create the usual rendering stuff.
renderer = vtk.vtkRenderer()
renWin = vtk.vtkRenderWindow()
renWin.AddRenderer(renderer)
iren = vtk.vtkRenderWindowInteractor()
iren.SetRenderWindow(renWin)
 
renderer.AddActor(wireActor)
renderer.SetBackground(1, 1, 1)
renderer.ResetCamera()
renderer.GetActiveCamera().Elevation(60.0)
renderer.GetActiveCamera().Azimuth(30.0)
renderer.GetActiveCamera().Zoom(1.0)
 
renWin.SetSize(300, 300)
 
# interact with data
renWin.Render()
iren.Start()

In [ ]:




import vtk
 
# create a rendering window and renderer
ren = vtk.vtkRenderer()
renWin = vtk.vtkRenderWindow()
renWin.AddRenderer(ren)
 
# create a renderwindowinteractor
iren = vtk.vtkRenderWindowInteractor()
iren.SetRenderWindow(renWin)
 
# create source
source = vtk.vtkPlaneSource()
source.SetCenter(1,0,0)
source.SetNormal(1,2,3)
 
# mapper
mapper = vtk.vtkPolyDataMapper()
mapper.SetInputConnection(source.GetOutputPort())
 
# actor
actor = vtk.vtkActor()
actor.SetMapper(mapper)
 
# assign actor to the renderer
ren.AddActor(actor)
 
# enable user interface interactor
iren.Initialize()
renWin.Render()
iren.Start()



In [ ]:

import vtk

# Create a superquadric
superquadricSource = vtk.vtkSuperquadricSource()
superquadricSource.SetPhiRoundness(3.1)
superquadricSource.SetThetaRoundness(1.0)
superquadricSource.Update()  # needed to GetBounds later

renderer = vtk.vtkRenderer()

mapper = vtk.vtkPolyDataMapper()
mapper.SetInputConnection(superquadricSource.GetOutputPort())

superquadricActor = vtk.vtkActor()
superquadricActor.SetMapper(mapper)

cubeAxesActor = vtk.vtkCubeAxesActor()
cubeAxesActor.SetBounds(superquadricSource.GetOutput().GetBounds())
cubeAxesActor.SetCamera(renderer.GetActiveCamera())
cubeAxesActor.GetTitleTextProperty(0).SetColor(1.0, 0.0, 0.0)
cubeAxesActor.GetLabelTextProperty(0).SetColor(1.0, 0.0, 0.0)

cubeAxesActor.GetTitleTextProperty(1).SetColor(0.0, 1.0, 0.0)
cubeAxesActor.GetLabelTextProperty(1).SetColor(0.0, 1.0, 0.0)

cubeAxesActor.GetTitleTextProperty(2).SetColor(0.0, 0.0, 1.0)
cubeAxesActor.GetLabelTextProperty(2).SetColor(0.0, 0.0, 1.0)

cubeAxesActor.DrawXGridlinesOn()
cubeAxesActor.DrawYGridlinesOn()
cubeAxesActor.DrawZGridlinesOn()
if vtk.VTK_MAJOR_VERSION > 5:
    cubeAxesActor.SetGridLineLocation(vtk.VTK_GRID_LINEAR)

cubeAxesActor.XAxisMinorTickVisibilityOff()
cubeAxesActor.YAxisMinorTickVisibilityOff()
cubeAxesActor.ZAxisMinorTickVisibilityOff()

renderer.AddActor(cubeAxesActor)
renderer.AddActor(superquadricActor)
renderer.GetActiveCamera().Azimuth(30)
renderer.GetActiveCamera().Elevation(30)

renderer.ResetCamera()

renderWindow = vtk.vtkRenderWindow()
renderWindow.AddRenderer(renderer)

renderWindowInteractor = vtk.vtkRenderWindowInteractor()
renderWindowInteractor.SetRenderWindow(renderWindow)

renderWindow.Render()
renderWindowInteractor.Start()

In [ ]:
###IMPORTANT SITES
#Installing Vtk: https://stackoverflow.com/questions/43184009/install-vtk-with-anaconda-3-6

#!/usr/bin/env python

# This example creates a polygonal model of a cone, and then renders
# it to the screen. It also defines an interaction style by creating a
# set of Command/Observers. (Note: it is far more efficient to create
# new styles by subclassing vtkInteractorStyle. This is just an
# illustrative example.) If you really want trackball behavior, look
# at the vtkInteractorStyleTrackballCamera class.

import vtk
import sys

# We create an instance of vtkConeSource and set some of its
# properties. The instance of vtkConeSource "cone" is part of a
# visualization pipeline (it is a source process object); it produces
# data (output type is vtkPolyData) which other filters may process.
#filename = "stl/Upper Limb Left.STL"
filename1= "stl/Forearm Left.STL"
filename2= "stl/Head.STL"

def actorInit(filename):
    reader = vtk.vtkSTLReader()
    reader.SetFileName(filename)
    mapper = vtk.vtkPolyDataMapper()
    if vtk.VTK_MAJOR_VERSION <= 5:
        mapper.SetInput(reader.GetOutput())
    else:
        mapper.SetInputConnection(reader.GetOutputPort())
 
    actor = vtk.vtkActor()
    actor.SetMapper(mapper)
    #actor.GetProperty().SetColor(0.5, 0.5, 1.0)

    return actor

In [ ]:
import vtk
import random
 
 
'''
There are two alternative ways to apply the transform.
 1) Use vtkTransformPolyDataFilter to create a new transformed polydata. 
    This method is useful if the transformed polydata is needed
      later in the pipeline
    To do this, set USER_MATRIX = True
 2) Apply the transform directly to the actor using vtkProp3D's SetUserMatrix. 
    No new data is produced.
    To do this, set USER_MATRIX = False
'''
USER_MATRIX = False
 
#Create an arrow.
arrowSource = vtk.vtkCylinderSource() #vtk.vtkArrowSource()
 
# Generate a random start and end point
random.seed(8775070)
startPoint = [0 for i in range(3)]
startPoint[0] = random.uniform(-10,10)
startPoint[1] = random.uniform(-10,10)
startPoint[2] = random.uniform(-10,10)
endPoint = [0 for i in range(3)]
endPoint[0] = random.uniform(-10,10)
endPoint[1] = random.uniform(-10,10)
endPoint[2] = random.uniform(-10,10)
 
# Compute a basis
normalizedX = [0 for i in range(3)]
normalizedY = [0 for i in range(3)]
normalizedZ = [0 for i in range(3)]
 
# The X axis is a vector from start to end
math = vtk.vtkMath()
math.Subtract(endPoint, startPoint, normalizedX)
length = math.Norm(normalizedX)
math.Normalize(normalizedX)
 
# The Z axis is an arbitrary vector cross X
arbitrary    = [0 for i in range(3)]
arbitrary[0] = random.uniform(-10,10)
arbitrary[1] = random.uniform(-10,10)
arbitrary[2] = random.uniform(-10,10)
math.Cross(normalizedX, arbitrary, normalizedZ)
math.Normalize(normalizedZ)
 
# The Y axis is Z cross X
math.Cross(normalizedZ, normalizedX, normalizedY)
matrix = vtk.vtkMatrix4x4()
 
# Create the direction cosine matrix
matrix.Identity()
for i in range(3):
  matrix.SetElement(i, 0, normalizedX[i])
  matrix.SetElement(i, 1, normalizedY[i])
  matrix.SetElement(i, 2, normalizedZ[i])
 
# Apply the transforms
transform = vtk.vtkTransform()
transform.Translate(startPoint)
transform.Concatenate(matrix)
transform.Scale(length, length, length)
 
# Transform the polydata
transformPD = vtk.vtkTransformPolyDataFilter()
transformPD.SetTransform(transform)
transformPD.SetInputConnection(arrowSource.GetOutputPort())
 
#Create a mapper and actor for the arrow
mapper = vtk.vtkPolyDataMapper()
actor = vtk.vtkActor()
 
if USER_MATRIX:
    mapper.SetInputConnection(arrowSource.GetOutputPort())
    actor.SetUserMatrix(transform.GetMatrix())
else:
    mapper.SetInputConnection(transformPD.GetOutputPort())
 
actor.SetMapper(mapper)
 
# Create spheres for start and end point
sphereStartSource = vtk.vtkSphereSource()
sphereStartSource.SetCenter(startPoint)
sphereStartMapper = vtk.vtkPolyDataMapper()
sphereStartMapper.SetInputConnection(sphereStartSource.GetOutputPort())
sphereStart = vtk.vtkActor()
sphereStart.SetMapper(sphereStartMapper)
sphereStart.GetProperty().SetColor(1.0, 1.0, .3)
 
sphereEndSource = vtk.vtkSphereSource()
sphereEndSource.SetCenter(endPoint)
sphereEndMapper = vtk.vtkPolyDataMapper()
sphereEndMapper.SetInputConnection(sphereEndSource.GetOutputPort())
sphereEnd = vtk.vtkActor()
sphereEnd.SetMapper(sphereEndMapper)
sphereEnd.GetProperty().SetColor(1.0, .3, .3)
 
#Create a renderer, render window, and interactor
renderer = vtk.vtkRenderer()
renderWindow = vtk.vtkRenderWindow()
renderWindow.AddRenderer(renderer)
renderWindowInteractor = vtk.vtkRenderWindowInteractor()
renderWindowInteractor.SetRenderWindow(renderWindow)
 
#Add the actor to the scene
renderer.AddActor(actor)
renderer.AddActor(sphereStart)
renderer.AddActor(sphereEnd)
renderer.SetBackground(.1, .2, .3) # Background color dark blue
 
 
#Render and interact
renderWindow.Render()
renderWindowInteractor.Start()

In [ ]:
import vtk

cone = vtk.vtkConeSource()
cone.SetHeight(3.0)
cone.SetRadius(1.0)
cone.SetResolution(10)

# In this example we terminate the pipeline with a mapper process
# object.  (Intermediate filters such as vtkShrinkPolyData could be
# inserted in between the source and the mapper.)  We create an
# instance of vtkPolyDataMapper to map the polygonal data into
# graphics primitives. We connect the output of the cone source to the
# input of this mapper.

coneMapper = vtk.vtkPolyDataMapper()
coneMapper.SetInputConnection(cone.GetOutputPort())

# Create an actor to represent the cone. The actor orchestrates
# rendering of the mapper's graphics primitives. An actor also refers
# to properties via a vtkProperty instance, and includes an internal
# transformation matrix. We set this actor's mapper to be coneMapper
# which we created above.

coneActor = vtk.vtkActor()
coneActor.SetMapper(coneMapper)

# Create the Renderer and assign actors to it. A renderer is like a
# viewport. It is part or all of a window on the screen and it is
# responsible for drawing the actors it has.  We also set the
# background color here.

ren = vtk.vtkRenderer()
ren.AddActor(coneActor)
#ren.AddActor(actorInit(filename))
#act1 = actorInit(filename1)
#ren.AddActor(act1)
#act2 = actorInit(filename2)
#ren.AddActor(act2)
ren.SetBackground(0.1, 0.2, 0.4)

# Finally we create the render window which will show up on the screen
# We put our renderer into the render window using AddRenderer. We
# also set the size to be 300 pixels by 300.
renWin = vtk.vtkRenderWindow()
renWin.AddRenderer(ren)
renWin.SetSize(500, 500)

# Define custom interaction.
iren = vtk.vtkRenderWindowInteractor()
iren.SetInteractorStyle(None)
iren.SetRenderWindow(renWin)

# Add the observers to watch for particular events. These invoke
# Python functions.
Rotating = 0
Panning = 0
Zooming = 0

# Handle the mouse button events.
def ButtonEvent(obj, event):
    global Rotating, Panning, Zooming
    if event == "LeftButtonPressEvent":
        Rotating = 1
        coneActor.RotateX(30)
        renWin.Render()
    elif event == "LeftButtonReleaseEvent":
        Rotating = 0
    elif event == "MiddleButtonPressEvent":
        Panning = 1
        coneActor.RotateZ(30)
        renWin.Render()
    elif event == "MiddleButtonReleaseEvent":
        Panning = 0
    elif event == "RightButtonPressEvent":
        Zooming = 1
        coneActor.RotateY(30)
        renWin.Render()
    elif event == "RightButtonReleaseEvent":
        Zooming = 0

# General high-level logic
def MouseMove(obj, event):
    global Rotating, Panning, Zooming
    global iren, renWin, ren
    lastXYpos = iren.GetLastEventPosition()
    lastX = lastXYpos[0]
    lastY = lastXYpos[1]

    xypos = iren.GetEventPosition()
    x = xypos[0]
    y = xypos[1]

    center = renWin.GetSize()
    centerX = center[0]/2.0
    centerY = center[1]/2.0
    #print(centerX,centerY)

    if Rotating:
        Rotate(ren, ren.GetActiveCamera(), x, y, lastX, lastY,
               centerX, centerY)
    elif Panning:
        Pan(ren, ren.GetActiveCamera(), x, y, lastX, lastY, centerX,
            centerY)
    elif Zooming:
        Dolly(ren, ren.GetActiveCamera(), x, y, lastX, lastY,
              centerX, centerY)


def Keypress(obj, event):
    key = obj.GetKeySym()
    if key == "e":
        obj.InvokeEvent("DeleteAllObjects")
        sys.exit()
    elif key == "w":
        Wireframe()
    elif key =="s":
        Surface()


# Routines that translate the events into camera motions.

# This one is associated with the left mouse button. It translates x
# and y relative motions into camera azimuth and elevation commands.
def Rotate(renderer, camera, x, y, lastX, lastY, centerX, centerY):
    camera.Azimuth(lastX-x)
    camera.Elevation(lastY-y)
    camera.OrthogonalizeViewUp()
    renWin.Render()


# Pan translates x-y motion into translation of the focal point and
# position.
def Pan(renderer, camera, x, y, lastX, lastY, centerX, centerY):
    FPoint = camera.GetFocalPoint()
    FPoint0 = FPoint[0]
    FPoint1 = FPoint[1]
    FPoint2 = FPoint[2]

    PPoint = camera.GetPosition()
    PPoint0 = PPoint[0]
    PPoint1 = PPoint[1]
    PPoint2 = PPoint[2]

    renderer.SetWorldPoint(FPoint0, FPoint1, FPoint2, 1.0)
    renderer.WorldToDisplay()
    DPoint = renderer.GetDisplayPoint()
    focalDepth = DPoint[2]

    APoint0 = centerX+(x-lastX)
    APoint1 = centerY+(y-lastY)

    renderer.SetDisplayPoint(APoint0, APoint1, focalDepth)
    renderer.DisplayToWorld()
    RPoint = renderer.GetWorldPoint()
    RPoint0 = RPoint[0]
    RPoint1 = RPoint[1]
    RPoint2 = RPoint[2]
    RPoint3 = RPoint[3]

    if RPoint3 != 0.0:
        RPoint0 = RPoint0/RPoint3
        RPoint1 = RPoint1/RPoint3
        RPoint2 = RPoint2/RPoint3

    camera.SetFocalPoint( (FPoint0-RPoint0)/2.0 + FPoint0,
                          (FPoint1-RPoint1)/2.0 + FPoint1,
                          (FPoint2-RPoint2)/2.0 + FPoint2)
    camera.SetPosition( (FPoint0-RPoint0)/2.0 + PPoint0,
                        (FPoint1-RPoint1)/2.0 + PPoint1,
                        (FPoint2-RPoint2)/2.0 + PPoint2)
    renWin.Render()


# Dolly converts y-motion into a camera dolly commands.
def Dolly(renderer, camera, x, y, lastX, lastY, centerX, centerY):
    dollyFactor = pow(1.02,(0.5*(y-lastY)))
    if camera.GetParallelProjection():
        parallelScale = camera.GetParallelScale()*dollyFactor
        camera.SetParallelScale(parallelScale)
    else:
        camera.Dolly(dollyFactor)
        renderer.ResetCameraClippingRange()

    renWin.Render()

# Wireframe sets the representation of all actors to wireframe.
def Wireframe():
    actors = ren.GetActors()
    actors.InitTraversal()
    actor = actors.GetNextItem()
    while actor:
        actor.GetProperty().SetRepresentationToWireframe()
        actor = actors.GetNextItem()

    renWin.Render()

# Surface sets the representation of all actors to surface.
def Surface():
    actors = ren.GetActors()
    actors.InitTraversal()
    actor = actors.GetNextItem()
    while actor:
        actor.GetProperty().SetRepresentationToSurface()
        actor = actors.GetNextItem()
    renWin.Render()


iren.AddObserver("LeftButtonPressEvent", ButtonEvent)
iren.AddObserver("LeftButtonReleaseEvent", ButtonEvent)
iren.AddObserver("MiddleButtonPressEvent", ButtonEvent)
iren.AddObserver("MiddleButtonReleaseEvent", ButtonEvent)
iren.AddObserver("RightButtonPressEvent", ButtonEvent)
iren.AddObserver("RightButtonReleaseEvent", ButtonEvent)
iren.AddObserver("MouseMoveEvent", MouseMove)
iren.AddObserver("KeyPressEvent", Keypress)


iren.Initialize()
renWin.Render()
iren.Start()

In [ ]:
ren = vtk.vtkRenderer()
ren.AddActor(coneActor)
#ren.AddActor(actorInit(filename))
#act1 = actorInit(filename1)
#ren.AddActor(act1)
#act2 = actorInit(filename2)
#ren.AddActor(act2)
ren.SetBackground(0.1, 0.2, 0.4)

# Finally we create the render window which will show up on the screen
# We put our renderer into the render window using AddRenderer. We
# also set the size to be 300 pixels by 300.
renWin = vtk.vtkRenderWindow()
renWin.AddRenderer(ren)
renWin.SetSize(300, 300)

# Define custom interaction.
iren = vtk.vtkRenderWindowInteractor()
iren.SetInteractorStyle(None)
iren.SetRenderWindow(renWin)



In [ ]:
from PyQt4 import QtCore
from PyQt4 import QtGui
from PyQt4 import uic
from vtk import *
import sys

# Create a PyQt window using a .ui file generated with Qt Designer ...
application = QtGui.QApplication(sys.argv)

window = uic.loadUi("gui.ui")
vertex_slider = window.findChild(QtGui.QSlider, "vertexCount")
edge_slider = window.findChild(QtGui.QSlider, "edgeCount")

# Create a simple pipeline source ...
source = vtkRandomGraphSource()
source.SetNumberOfVertices(vertex_slider.value())
source.SetNumberOfEdges(edge_slider.value())
source.SetStartWithTree(True)

# Define event-handlers that update the pipeline in response to widget events ...
def change_vertex_count(count):
  source.SetNumberOfVertices(count)
  edge_slider.setValue(source.GetNumberOfEdges())
  view.GetRenderer().ResetCamera()
  render_window.Render()

def change_edge_count(count):
  source.SetNumberOfEdges(count)
  edge_slider.setValue(source.GetNumberOfEdges())
  view.GetRenderer().ResetCamera()
  render_window.Render()

# Connect the GUI widgets to the event-handlers ...
QtCore.QObject.connect(vertex_slider, QtCore.SIGNAL("valueChanged(int)"), change_vertex_count)
QtCore.QObject.connect(edge_slider, QtCore.SIGNAL("valueChanged(int)"), change_edge_count)
window.show()

# Setup a VTK view ...
view = vtkGraphLayoutView()
view.AddRepresentationFromInputConnection(source.GetOutputPort())

view.GetRenderWindow().SetSize(600, 600)

# This initializes the VTK window for interaction, but doesn't start an event-loop ...
view.GetRenderWindow().Start()

# Start the Qt event-loop ...
application.exec_()

In [ ]:
%gui qt

import vtk
 
filename = "final_skull_visible_female_full_decimated_smoothed.stl"
filename1 = "files/Left_Radius_Ulna.stl"
filename2 = "files/Humerus_Artic_2-part.stl"
filename3 = "files/Left_Carpus.stl"
filename4 = "files/Left_Clavicle.stl"
filename5 = "files/Left_Clavicle_Artic.stl"
filename6 = "files/Left_Humerus.stl"
filename7 = "files/Left_Humerus_Artic.stl"
filename8 = "files/Left_MC_Phalanges.stl"
filename9 = "files/Left_Radius_Ulna.stl"
filename10 = "files/Left_Radius__Ulna_Artic.stl"
filename11 = "files/Left_Scapula.stl"
filename12 = "files/Left_Scapula_Artic.stl"

def actorInit(filename):
    reader = vtk.vtkSTLReader()
    reader.SetFileName(filename)
    mapper = vtk.vtkPolyDataMapper()
    if vtk.VTK_MAJOR_VERSION <= 5:
        mapper.SetInput(reader.GetOutput())
    else:
        mapper.SetInputConnection(reader.GetOutputPort())
 
    actor = vtk.vtkActor()
    actor.SetMapper(mapper)
    #actor.GetProperty().SetColor(0.5, 0.5, 1.0)

    return actor
 
# Create a rendering window and renderer
ren = vtk.vtkRenderer()
renWin = vtk.vtkRenderWindow()
renWin.AddRenderer(ren)
 
# Create a renderwindowinteractor
iren = vtk.vtkRenderWindowInteractor()
iren.SetRenderWindow(renWin)
 
# Assign actor to the renderer
ren.AddActor(actorInit(filename))
ren.AddActor(actorInit(filename2))
ren.AddActor(actorInit(filename3))
ren.AddActor(actorInit(filename4))
ren.AddActor(actorInit(filename5))
ren.AddActor(actorInit(filename6))
ren.AddActor(actorInit(filename7))
ren.AddActor(actorInit(filename8))
ren.AddActor(actorInit(filename9))
ren.AddActor(actorInit(filename10))
ren.AddActor(actorInit(filename11))
ren.AddActor(actorInit(filename12))
 
# Enable user interface interactor
iren.Initialize()
renWin.Render()
iren.Start()
print("rahul")

In [ ]:
import vtk
 
# create two cones, rotate one of them.
 
# create a rendering window and renderer
ren = vtk.vtkRenderer()
renWin = vtk.vtkRenderWindow()
renWin.AddRenderer(ren)
WIDTH=640
HEIGHT=480
renWin.SetSize(WIDTH,HEIGHT)
 
# create a renderwindowinteractor
iren = vtk.vtkRenderWindowInteractor()
iren.SetRenderWindow(renWin)
 
# create cone
source = vtk.vtkConeSource()
source.SetResolution(60)
source.SetCenter(-2,0,0)
 
# create a transform that rotates the cone
transform = vtk.vtkTransform()
transform.RotateWXYZ(45,0,1,0)
transformFilter=vtk.vtkTransformPolyDataFilter()
transformFilter.SetTransform(transform)
transformFilter.SetInputConnection(source.GetOutputPort())
transformFilter.Update()
 
# mapper for original cone
coneMapper1 = vtk.vtkPolyDataMapper()
if vtk.VTK_MAJOR_VERSION <= 5:
    coneMapper1.SetInput(source.GetOutput())
else:
    coneMapper1.SetInputConnection(source.GetOutputPort())
 
 
# another mapper for the rotated cone
coneMapper2 = vtk.vtkPolyDataMapper()
if vtk.VTK_MAJOR_VERSION <= 5:
    coneMapper2.SetInput(transformFilter.GetOutput())
else:
    coneMapper2.SetInputConnection(transformFilter.GetOutputPort())
 
# actor for original cone
actor1 = vtk.vtkActor()
actor1.SetMapper(coneMapper1)
 
# actor for rotated cone
actor2 = vtk.vtkActor()
actor2.SetMapper(coneMapper2)
 
 
# color the original cone red
actor1.GetProperty().SetColor(1,0,0) # (R,G,B)
# color rotated cone blue
actor2.GetProperty().SetColor(0,0,1) # (R,G,B)
 
# assign actor to the renderer
ren.AddActor(actor1)
ren.AddActor(actor2)
 
# enable user interface interactor
iren.Initialize()
renWin.Render()
iren.Start()

In [ ]:
import vtk
from numpy import random

class VtkPointCloud:

    def __init__(self, zMin=-10.0, zMax=10.0, maxNumPoints=1e6):
        self.maxNumPoints = maxNumPoints
        self.vtkPolyData = vtk.vtkPolyData()
        self.clearPoints()
        mapper = vtk.vtkPolyDataMapper()
        mapper.SetInputData(self.vtkPolyData)
        mapper.SetColorModeToDefault()
        mapper.SetScalarRange(zMin, zMax)
        mapper.SetScalarVisibility(1)
        self.vtkActor = vtk.vtkActor()
        self.vtkActor.SetMapper(mapper)

    def addPoint(self, point):
        if self.vtkPoints.GetNumberOfPoints() < self.maxNumPoints:
            pointId = self.vtkPoints.InsertNextPoint(point[:])
            self.vtkDepth.InsertNextValue(point[2])
            self.vtkCells.InsertNextCell(1)
            self.vtkCells.InsertCellPoint(pointId)
        else:
            r = random.randint(0, self.maxNumPoints)
            self.vtkPoints.SetPoint(r, point[:])
        self.vtkCells.Modified()
        self.vtkPoints.Modified()
        self.vtkDepth.Modified()

    def clearPoints(self):
        self.vtkPoints = vtk.vtkPoints()
        self.vtkCells = vtk.vtkCellArray()
        self.vtkDepth = vtk.vtkDoubleArray()
        self.vtkDepth.SetName('DepthArray')
        self.vtkPolyData.SetPoints(self.vtkPoints)
        self.vtkPolyData.SetVerts(self.vtkCells)
        self.vtkPolyData.GetPointData().SetScalars(self.vtkDepth)
        self.vtkPolyData.GetPointData().SetActiveScalars('DepthArray')

pointCloud = VtkPointCloud()
for k in range(1000):
    point = 20*(random.rand(3)-0.5)
    pointCloud.addPoint(point)
pointCloud.addPoint([0,0,0])
pointCloud.addPoint([0,0,0])
pointCloud.addPoint([0,0,0])
pointCloud.addPoint([0,0,0])

# Renderer
renderer = vtk.vtkRenderer()
renderer.AddActor(pointCloud.vtkActor)
renderer.SetBackground(.2, .3, .4)
renderer.ResetCamera()

# Render Window
renderWindow = vtk.vtkRenderWindow()
renderWindow.AddRenderer(renderer)

# Interactor
renderWindowInteractor = vtk.vtkRenderWindowInteractor()
renderWindowInteractor.SetRenderWindow(renderWindow)

# Begin Interaction
renderWindow.Render()
renderWindowInteractor.Start()

In [ ]:
#!/usr/bin/env python

# Demonstrate how to use the vtkBoxWidget to translate, scale, and
# rotate actors.  The basic idea is that the box widget controls an
# actor's transform. A callback which modifies the transform is
# invoked as the box widget is manipulated.

import vtk

# Start by creating some simple geometry; in this case a mace.
sphere = vtk.vtkSphereSource()
cone = vtk.vtkConeSource()
glyph = vtk.vtkGlyph3D()
glyph.SetInputConnection(sphere.GetOutputPort())
glyph.SetSourceConnection(cone.GetOutputPort())
glyph.SetVectorModeToUseNormal()
glyph.SetScaleModeToScaleByVector()
glyph.SetScaleFactor(0.25)
appendData = vtk.vtkAppendPolyData()
appendData.AddInputConnection(glyph.GetOutputPort())
appendData.AddInputConnection(sphere.GetOutputPort())
maceMapper = vtk.vtkPolyDataMapper()
maceMapper.SetInputConnection(appendData.GetOutputPort())
maceActor = vtk.vtkLODActor()
maceActor.SetMapper(maceMapper)
maceActor.VisibilityOn()

# Create the RenderWindow, Renderer and both Actors
ren = vtk.vtkRenderer()
renWin = vtk.vtkRenderWindow()
renWin.AddRenderer(ren)
iren = vtk.vtkRenderWindowInteractor()
iren.SetRenderWindow(renWin)

# The box widget observes the events invoked by the render window
# interactor.  These events come from user interaction in the render
# window.
boxWidget = vtk.vtkBoxWidget()
boxWidget.SetInteractor(iren)
boxWidget.SetPlaceFactor(1.25)

# Add the actors to the renderer, set the background and window size.
ren.AddActor(maceActor)
ren.SetBackground(0.1, 0.2, 0.4)
renWin.SetSize(300, 300)

# As the box widget is interacted with, it produces a transformation
# matrix that is set on the actor.
t = vtk.vtkTransform()
def TransformActor(obj, event):
    global t, maceActor
    obj.GetTransform(t)
    maceActor.SetUserTransform(t)

# Place the interactor initially. The actor is used to place and scale
# the interactor. An observer is added to the box widget to watch for
# interaction events. This event is captured and used to set the
# transformation matrix of the actor.
boxWidget.SetProp3D(maceActor)
boxWidget.PlaceWidget()
#boxWidget.AddObserver("InteractionEvent", TransformActor)

iren.Initialize()
renWin.Render()
iren.Start()

In [ ]:
import vtk

# our data source, a PolyDataReader
reader = vtk.vtkPolyDataReader()
reader.SetFileName("sphere.vtk")
reader.Update() # by calling Update() we read the file

# find the range of the point scalars
a,b = reader.GetOutput().GetPointData().GetScalars().GetRange()
nm =  reader.GetOutput().GetPointData().GetScalars().GetName()

# show how to print a string in python, it is similar to C-style sprintf
print ("Range of %s: %4.2f-%4.2f" %(nm,a,b))

# transfer function (lookup table) for mapping point scalar data
# to colors (parent class is vtkScalarsToColors)
lut = vtk.vtkColorTransferFunction()
lut.AddRGBPoint(a,         0.0, 0.0, 1.0)
lut.AddRGBPoint(a+(b-a)/4, 0.0, 0.5, 0.5)
lut.AddRGBPoint(a+(b-a)/2, 0.0, 1.0, 0.0)
lut.AddRGBPoint(b-(b-a)/4, 0.5, 0.5, 0.0)
lut.AddRGBPoint(b,         1.0, 0.0, 0.0)

# the mapper that will use the lookup table 
mapper = vtk.vtkPolyDataMapper()
mapper.SetLookupTable(lut)
mapper.SetScalarRange(a,b)
mapper.SetInput(reader.GetOutput()) # connection
# important! tell which data you want to use
# here we use the point data
mapper.SetScalarModeToUsePointData() 

# the actor
myActor = vtk.vtkActor()
myActor.SetMapper( mapper )

# a colorbar to display the colormap
scalarBar = vtk.vtkScalarBarActor()
scalarBar.SetLookupTable( mapper.GetLookupTable() )
scalarBar.SetTitle("Point scalar value")
scalarBar.SetOrientationToHorizontal()
scalarBar.GetLabelTextProperty().SetColor(0,0,1)
scalarBar.GetTitleTextProperty().SetColor(0,0,1)

# position it in window
coord = scalarBar.GetPositionCoordinate()
coord.SetCoordinateSystemToNormalizedViewport()
coord.SetValue(0.1,0.05)
scalarBar.SetWidth(.8)
scalarBar.SetHeight(.15)

# renderer and render window 
ren = vtk.vtkRenderer()
ren.SetBackground(1, 1, 1)
renWin = vtk.vtkRenderWindow()
renWin.SetSize(512, 512)
renWin.AddRenderer( ren )

# render window interactor
iren = vtk.vtkRenderWindowInteractor()
iren.SetRenderWindow( renWin )

# add the actors to the renderer
ren.AddActor( myActor )
ren.AddActor( scalarBar )

# render
renWin.Render()

# initialize and start the interactor
iren.Initialize()
iren.Start()
